In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import spline # Spline interpolation to smooth curves

from os.path import join as pjoin
import os
import pickle # Access and serialize objects
import kde # Kernel Density Estimation functions

In [ ]:
img_dir = pjoin('..', 'images', 'tourn_8')
pop_dir = pjoin('..', 'pop_data', 'tourn_8')

lr = 'r'

tups1 = [(1, ''), (1, '_sel'), (1, '_cross'), (2, ''), (2, '_sel')]
tups2 = [(1, ''), (10, ''), (20, ''), (30, '')]

#sufix = ['', '_sel', '_cross']
d = {
     1: ['', '_sel', '_cross'],
     2: ['', '_sel'],
    10: [''],
    20: [''],
    30: ['']
}
basename = '%02d' + lr + '_100%s'
names = [basename % (k, v) for k in [1,2,10,20,30] for v in d[k]]
print names

# Add directory name and .dat sufix
fnames = {}
for name in names:
    fnames[name] = pjoin(pop_dir, name + '.dat')

print '\n'.join('%s\n%s\n' % (k,v) for k,v in fnames.iteritems())

# Reading fitness from generations

In [ ]:
# Reading population fitness
fit_dict = dict()
for k,v in fnames.iteritems():
    with open(v, 'rb') as pic_file:
        _ = pickle.load(pic_file)
        fitness = np.array(pickle.load(pic_file))
    fitness = fitness[fitness!=0] # remove 0s
    fitness[::-1].sort()
    fit_dict[k] = fitness / 100

# Defining some helper functions

In [ ]:
def smooth(x, y):
    x_sm = np.array(x)
    y_sm = np.array(y)
    x_smooth = np.linspace(x_sm.min(), x_sm.max(), 100000)
    y_smooth = spline(x, y, x_smooth)
    return x_smooth, y_smooth

from scipy import interpolate
def smooth2(x, y):
    x_sm = np.array(x)
    y_sm = np.array(y)
    
    tck = interpolate.splrep(x, y, s=0)
    xnew = np.linspace(x_sm.min(), x_sm.max(), 100000)
    ynew = interpolate.splev(xnew, tck, der=0)
    
    return xnew, ynew

def plot_histogram(fitness, bins = 'auto', color ='b', transp = 0.6, plot = True):    
    if plot:
        y, bins, _ = plt.hist(fitness, bins=bins, normed=True, facecolor=color, alpha=transp)
    else:
        y, bins = np.histogram(fitness, bins, normed=True)
        
    return {'y': y, 'bins': bins}

def plot_hist_line(x, y, color = 'b', interp = True, label = None):
    x_center = 0.5*(x[1:] + x[:-1])
    if interp:
        x_center, y = smooth(x_center, y)
        marker = None
    else:
        marker = 'o'
    plt.plot(x_center, y, marker = marker, color=color, linewidth = 2, label = label)
    
def plot_kde(x, y, color = 'b', bw = 2, interp = True, label = None):
    x_center = 0.5*(x[1:] + x[:-1])
    y = kde.kde_scipy(y, x_center, bw) 
    if interp:
        x_center, y = smooth(x_center, y)
        marker = None
    else:
        marker = 'o'
    plt.plot(x_center, y, marker = marker, color=color, linewidth = 2, label = label)

# Testing our plots

In [ ]:
fitness = fit_dict[basename % (1, '')]
histo = plot_histogram(fitness, plot = True)

plot_hist_line(histo['bins'], histo['y'], interp = False, color = 'black')
plot_kde(histo['bins'], fitness, bw = 0.5, interp = False, color = 'red')

# Get the horizontal limits for the comparison graphs

In [ ]:
xlim = []

x_min, x_max = 100000, 0
for n in names[:-3]:
    x_min = min(x_min, fit_dict[n].min())
    x_max = max(x_max, fit_dict[n].max())
xlim.append([x_min-5, x_max+5])
    
x_min, x_max = fit_dict[basename%(1,'')].min(), fit_dict[basename%(1,'')].max()
for n in names[-3:]:
    x_min = min(x_min, fit_dict[n].min())
    x_max = max(x_max, fit_dict[n].max())
xlim.append([x_min-5, x_max+5])

print xlim

# Plotting the comparison of the first generations

In [ ]:
seed = False

legends = [
    '1st gen',
    '1st + Sel',
    '1st + Cross',
    '2nd gen',
    '2nd + Sel',
    '10th gen',
    '20th gen',
    '30th gen',
]
legends = {v: legends[i] for i,v in enumerate(names) }
print legends

colors = ['green', 'red', 'cyan', 'orange']
for i,name in enumerate(names[1:-3]):    
    fitness = fit_dict[basename % (1, '')]
    histo = plot_histogram(fitness, plot = False, transp = 0.3)
    line = plot_hist_line(histo['bins'], histo['y'], interp = False, label = '1st gen')

    fitness = fit_dict[name]
    histo = plot_histogram(fitness, bins=histo['bins'], plot = True, color = colors[i-1])
    line = plot_hist_line(histo['bins'], histo['y'], interp = False, color = colors[i-1], label = legends[name])
    
    if seed == True:
        aux = plt.axvline(x=104.0829, color='black', linewidth=3, label = 'Seed')

    plt.legend(loc=2)

    plt.grid()
    plt.title('Comparison of different generations')
    plt.ylabel('Probability')
    plt.xlabel('Fitness ($)')
    
    # Set labels
    ticks = np.arange(55, 201, 5)
    plt.xticks(ticks, ticks*100, rotation=60)
    
    axes = plt.gca()
    #axes.set_xlim(xlim[0])
    axes.set_xlim([79, 116])
    axes.set_ylim([0,0.3])
    
    #print "Saving at", pjoin(img_dir, name + '.pdf')
    plt.show()

# Plotting the comparison of the first generation to the last tenths

In [ ]:
for i,name in enumerate(names[-3:]):    
    fitness = fit_dict[basename % (1, '')]
    histo = plot_histogram(fitness, plot = True, transp = 0.3)
    line = plot_hist_line(histo['bins'], histo['y'], interp = False, label = '1st gen')

    fitness = fit_dict[name]
    histo = plot_histogram(fitness, bins=10-i*2, plot = True, color = 'm')
    line = plot_hist_line(histo['bins'], histo['y'], interp = False, color = 'm', label = legends[name])
    
    if seed == True:
        aux = plt.axvline(x=104.0829, color='black', linewidth=3, label = 'Seed')

    plt.legend(loc=2)

    plt.grid()
    plt.title('Comparison of different generations')
    plt.ylabel('Probability')
    plt.xlabel('Fitness ($)')
    
    # Set labels
    ticks = np.arange(55, 201, 5)
    plt.xticks(ticks, ticks*100, rotation=60)
    
    axes = plt.gca()
    #axes.set_xlim(xlim[1])
    axes.set_xlim([79, 136])
    axes.set_ylim([0,0.65])
    
    #print "Saving at", pjoin(img_dir, name + '.pdf')
    plt.show()

In [ ]:
for n in names:
    plt.plot(fit_dict[n], label = n)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()
